In [1]:
import numpy as np
import csv, gzip, os, sys, gc
import math
import torch
from torch import nn
import torch.optim as optim
from torch.nn import functional as F

import logging
import datetime
import optparse
import pandas as pd
import os
from sklearn.metrics import log_loss
import ast
from torch.utils.data import Dataset
from sklearn.metrics import log_loss
from torch.utils.data import DataLoader
from scipy.ndimage import uniform_filter
from torch.optim.lr_scheduler import StepLR

from apex.parallel import DistributedDataParallel as DDP
from apex.fp16_utils import *
from apex import amp, optimizers
from apex.multi_tensor_apply import multi_tensor_applier


In [6]:

# Print info about environments
parser = optparse.OptionParser()
parser.add_option('-s', '--seed', action="store", dest="seed", help="model seed", default="1234")
parser.add_option('-o', '--fold', action="store", dest="fold", help="Fold for split", default="0")
parser.add_option('-p', '--nbags', action="store", dest="nbags", help="Number of bags for averaging", default="4")
parser.add_option('-e', '--epochs', action="store", dest="epochs", help="epochs", default="10")
parser.add_option('-b', '--batchsize', action="store", dest="batchsize", help="batch size", default="4")
parser.add_option('-r', '--rootpath', action="store", dest="rootpath", help="root directory", default="")
parser.add_option('-i', '--imgpath', action="store", dest="imgpath", help="root directory", default="data/mount/512X512X6/")
parser.add_option('-w', '--workpath', action="store", dest="workpath", help="Working path", default="densenetv1/weights")
parser.add_option('-f', '--weightsname', action="store", dest="weightsname", help="Weights file name", default="pytorch_model.bin")
parser.add_option('-l', '--lr', action="store", dest="lr", help="learning rate", default="0.00005")
parser.add_option('-g', '--logmsg', action="store", dest="logmsg", help="root directory", default="Recursion-pytorch")
parser.add_option('-c', '--size', action="store", dest="size", help="model size", default="480")
parser.add_option('-a', '--globalepoch', action="store", dest="globalepoch", help="root directory", default="3")
parser.add_option('-n', '--loadcsv', action="store", dest="loadcsv", help="Convert csv embeddings to numpy", default="F")
parser.add_option('-j', '--lstm_units', action="store", dest="lstm_units", help="Lstm units", default="128")
parser.add_option('-d', '--dropout', action="store", dest="dropout", help="LSTM input spatial dropout", default="0.3")
parser.add_option('-z', '--decay', action="store", dest="decay", help="Weight Decay", default="0.0")
parser.add_option('-m', '--lrgamma', action="store", dest="lrgamma", help="Scheduler Learning Rate Gamma", default="1.0")
parser.add_option('-k', '--ttahflip', action="store", dest="ttahflip", help="Bag with horizontal flip on and off", default="F")
parser.add_option('-q', '--ttatranspose', action="store", dest="ttatranspose", help="Bag with horizontal flip on and off", default="F")
parser.add_option('-x', '--datapath', action="store", dest="datapath", help="Data path", default="data")


options, args = parser.parse_args()
package_dir = options.rootpath
sys.path.append(package_dir)
sys.path.insert(0, 'scripts')
from logs import get_logger
from utils import dumpobj, loadobj, GradualWarmupScheduler

options.logmsg = "Rsna-lstm-0-1-fp16"
options.epochs = 12
options.fold = 1
options.lr = 0.00001
options.batchsize = 4
options.workpath = '/bin/resnext101v12fold1'
options.datapath = '/bin/resnext101v12fold1'
options.lrgamma = 0.95
options.nbags = 12
options.globalepoch = 0
options.lstm_units = 2048
options.size = 480

In [7]:
# Print info about environments
logger = get_logger(options.logmsg, 'INFO') # noqa
logger.info('Cuda set up : time {}'.format(datetime.datetime.now().time()))

device=torch.device('cuda')
logger.info('Device : {}'.format(torch.cuda.get_device_name(0)))
logger.info('Cuda available : {}'.format(torch.cuda.is_available()))
n_gpu = torch.cuda.device_count()
logger.info('Cuda n_gpus : {}'.format(n_gpu ))

logger.info('Load params : time {}'.format(datetime.datetime.now().time()))
for (k,v) in options.__dict__.items():
    logger.info('{}{}'.format(k.ljust(20), v))
    
SEED = int(options.seed)
SIZE = int(options.size)
EPOCHS = int(options.epochs)
GLOBALEPOCH=int(options.globalepoch)
n_epochs = EPOCHS 
lr=float(options.lr)
lrgamma=float(options.lrgamma)
DECAY=float(options.decay)
batch_size = int(options.batchsize)
ROOT = options.rootpath
path_data = os.path.join(ROOT, options.datapath)
WORK_DIR = os.path.join(ROOT, options.workpath)
path_emb = os.path.join(ROOT, options.workpath)

WEIGHTS_NAME = options.weightsname
fold = int(options.fold)
LOADCSV= options.loadcsv=='T'
LSTM_UNITS=int(options.lstm_units)
nbags=int(options.nbags)
DROPOUT=float(options.dropout)
TTAHFLIP= 'T' if options.ttahflip=='T' else ''
TTATRANSPOSE= 'P' if options.ttatranspose=='T' else ''
n_classes = 6
label_cols = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']


2020-04-29 09:38:33,519 - Rsna-lstm-0-1-fp16 - INFO - Cuda set up : time 09:38:33.519203
2020-04-29 09:38:33,519 - Rsna-lstm-0-1-fp16 - INFO - Cuda set up : time 09:38:33.519203
2020-04-29 09:38:33,521 - Rsna-lstm-0-1-fp16 - INFO - Device : GeForce GTX 1080 Ti
2020-04-29 09:38:33,521 - Rsna-lstm-0-1-fp16 - INFO - Device : GeForce GTX 1080 Ti
2020-04-29 09:38:33,525 - Rsna-lstm-0-1-fp16 - INFO - Cuda available : True
2020-04-29 09:38:33,525 - Rsna-lstm-0-1-fp16 - INFO - Cuda available : True
2020-04-29 09:38:33,526 - Rsna-lstm-0-1-fp16 - INFO - Cuda n_gpus : 5
2020-04-29 09:38:33,526 - Rsna-lstm-0-1-fp16 - INFO - Cuda n_gpus : 5
2020-04-29 09:38:33,528 - Rsna-lstm-0-1-fp16 - INFO - Load params : time 09:38:33.528839
2020-04-29 09:38:33,528 - Rsna-lstm-0-1-fp16 - INFO - Load params : time 09:38:33.528839
2020-04-29 09:38:33,533 - Rsna-lstm-0-1-fp16 - INFO - seed                1234
2020-04-29 09:38:33,533 - Rsna-lstm-0-1-fp16 - INFO - seed                1234
2020-04-29 09:38:33,534 - Rs

In [4]:
def makeSub(ypred, imgs):
    imgls = np.array(imgs).repeat(len(label_cols)) 
    icdls = pd.Series(label_cols*ypred.shape[0])   
    yidx = ['{}_{}'.format(i,j) for i,j in zip(imgls, icdls)]
    subdf = pd.DataFrame({'ID' : yidx, 'Label': ypred.flatten()})
    return subdf

class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

def criterion(data, targets, criterion = torch.nn.BCEWithLogitsLoss()):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    loss_all = criterion(data, targets)
    loss_any = criterion(data[:,-1:], targets[:,-1:])
    return (loss_all*6 + loss_any*1)/7

class IntracranialDataset(Dataset):
    def __init__(self, df, mat, labels=label_cols):
        self.data = df
        self.mat = mat
        self.labels = labels
        self.patients = df.SliceID.unique()
        self.data = self.data.set_index('SliceID')

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        
        patidx = self.patients[idx]
        patdf = self.data.loc[patidx].sort_values('seq')
        patemb = self.mat[patdf['embidx'].values]

        patdeltalag  = np.zeros(patemb.shape)
        patdeltalead = np.zeros(patemb.shape)
        patdeltalag [1:] = patemb[1:]-patemb[:-1]
        patdeltalead[:-1] = patemb[:-1]-patemb[1:]

        patemb = np.concatenate((patemb, patdeltalag, patdeltalead), -1)
        
        ids = torch.tensor(patdf['embidx'].values)

        if self.labels:
            labels = torch.tensor(patdf[label_cols].values)
            return {'emb': patemb, 'embidx' : ids, 'labels': labels}    
        else:      
            return {'emb': patemb, 'embidx' : ids}

def predict(loader):
    valls = []
    imgls = []
    imgdf = loader.dataset.data.reset_index().set_index('embidx')[['Image']].copy()
    for step, batch in enumerate(loader):
        inputs = batch["emb"]
        mask = batch['mask'].to(device, dtype=torch.int)
        inputs = inputs.to(device, dtype=torch.float)
        logits = model(inputs)
        # get the mask for masked labels
        maskidx = mask.view(-1)==1
        # reshape for
        logits = logits.view(-1, n_classes)[maskidx]
        valls.append(torch.sigmoid(logits).detach().cpu().numpy())
        # Get the list of images
        embidx = batch["embidx"].detach().cpu().numpy().astype(np.int32)
        embidx = embidx.flatten()[embidx.flatten()>-1]
        images = imgdf.loc[embidx].Image.tolist() 
        imgls += images
    return np.concatenate(valls, 0), imgls

In [5]:
# Print info about environments
logger.info('Cuda set up : time {}'.format(datetime.datetime.now().time()))

# Get image sequences
trnmdf = pd.read_csv(os.path.join(path_data, 'train_metadata.csv.gz'))
tstmdf = pd.read_csv(os.path.join(path_data, 'test_metadata.csv.gz'))

trnmdf['SliceID'] = trnmdf[['PatientID', 'SeriesInstanceUID', 'StudyInstanceUID']].apply(lambda x: '{}__{}__{}'.format(*x.tolist()), 1)
tstmdf['SliceID'] = tstmdf[['PatientID', 'SeriesInstanceUID', 'StudyInstanceUID']].apply(lambda x: '{}__{}__{}'.format(*x.tolist()), 1)

poscols = ['ImagePos{}'.format(i) for i in range(1, 4)]
trnmdf[poscols] = pd.DataFrame(trnmdf['ImagePositionPatient']\
              .apply(lambda x: list(map(float, ast.literal_eval(x)))).tolist())
tstmdf[poscols] = pd.DataFrame(tstmdf['ImagePositionPatient']\
              .apply(lambda x: list(map(float, ast.literal_eval(x)))).tolist())

trnmdf['seq'] = (trnmdf.groupby(['SliceID']).cumcount() + 1)
tstmdf['seq'] = (tstmdf.groupby(['SliceID']).cumcount() + 1)

keepcols = ['PatientID', 'SliceID', 'SOPInstanceUID', 'seq']
trnmdf = trnmdf[keepcols]
tstmdf = tstmdf[keepcols]

trnmdf.columns = tstmdf.columns = ['PatientID', 'SliceID', 'Image', 'seq']

2020-04-28 17:07:46,074 - Rsna-lstm-0-1-fp16 - INFO - Cuda set up : time 17:07:46.074394


In [9]:
# Load Data Frames
trndf = loadobj(os.path.join(path_emb, 'loader_trn_size{}_fold{}_ep{}'.format(SIZE, fold, GLOBALEPOCH))).dataset.data
valdf = loadobj(os.path.join(path_emb, 'loader_val_size{}_fold{}_ep{}'.format(SIZE, fold, GLOBALEPOCH))).dataset.data
tstdf = loadobj(os.path.join(path_emb, 'loader_tst_size{}_fold{}_ep{}'.format(SIZE, fold, GLOBALEPOCH))).dataset.data

trndf['embidx'] = range(trndf.shape[0])
valdf['embidx'] = range(valdf.shape[0])
tstdf['embidx'] = range(tstdf.shape[0])

trndf = trndf.merge(trnmdf.drop('PatientID', 1), on = 'Image')
valdf = valdf.merge(trnmdf.drop('PatientID', 1), on = 'Image')
tstdf = tstdf.merge(tstmdf, on = 'Image')

logger.info('Trn df shape {} {}'.format(*trndf.shape))
logger.info('Val df shape {} {}'.format(*valdf.shape))
logger.info('Tst df shape {} {}'.format(*tstdf.shape))




2020-04-28 17:16:54,131 - Rsna-lstm-0-1-fp16 - INFO - Trn df shape 539827 12
2020-04-28 17:16:54,132 - Rsna-lstm-0-1-fp16 - INFO - Val df shape 134430 12
2020-04-28 17:16:54,133 - Rsna-lstm-0-1-fp16 - INFO - Tst df shape 78545 6


In [6]:
# Load embeddings
embnm='emb_sz256_wt256_fold{}_epoch{}'.format(fold, GLOBALEPOCH)
logger.info('Load npy..')

def loademb(TYPE, SIZE, fold, GLOBALEPOCH, TTA=''):
    return np.load(os.path.join(path_emb, 'emb{}_{}_size{}_fold{}_ep{}.npz'.format(TTA, TYPE, SIZE, fold, GLOBALEPOCH)))['arr_0']

logger.info('Load embeddings...')
trnembls = [loademb('trn', SIZE, fold, GLOBALEPOCH)]
valembls = [loademb('val', SIZE, fold, GLOBALEPOCH)]
tstembls = [loademb('tst', SIZE, fold, GLOBALEPOCH)]


2020-04-28 17:08:34,999 - Rsna-lstm-0-1-fp16 - INFO - Load npy..
2020-04-28 17:08:35,002 - Rsna-lstm-0-1-fp16 - INFO - Load embeddings...


In [7]:
if TTAHFLIP=='T':
    logger.info('Load hflip...')
    trnembls.append(loademb('trn', SIZE, fold, GLOBALEPOCH, TTA='T'))
    valembls.append(loademb('val', SIZE, fold, GLOBALEPOCH, TTA='T'))
    tstembls.append(loademb('tst', SIZE, fold, GLOBALEPOCH, TTA='T'))
if TTATRANSPOSE=='P':
    logger.info('Load transpose...')
    trnembls.append(loademb('trn', SIZE, fold, GLOBALEPOCH, TTA='P'))
    valembls.append(loademb('val', SIZE, fold, GLOBALEPOCH, TTA='P'))
    tstembls.append(loademb('tst', SIZE, fold, GLOBALEPOCH, TTA='P'))

trnemb = sum(trnembls)/len(trnembls)
valemb = sum(valembls)/len(valembls)
tstemb = sum(tstembls)/len(tstembls)

logger.info('Trn shape {} {}'.format(*trnemb.shape))
logger.info('Val shape {} {}'.format(*valemb.shape))
logger.info('Tst shape {} {}'.format(*tstemb.shape))
logger.info('Add stg1 test labels to train')
del trnembls, valembls, tstembls
gc.collect()

2020-04-28 17:15:07,208 - Rsna-lstm-0-1-fp16 - INFO - Trn shape 539827 2048
2020-04-28 17:15:07,211 - Rsna-lstm-0-1-fp16 - INFO - Val shape 134430 2048
2020-04-28 17:15:07,212 - Rsna-lstm-0-1-fp16 - INFO - Tst shape 78545 2048
2020-04-28 17:15:07,213 - Rsna-lstm-0-1-fp16 - INFO - Add stg1 test labels to train


120

In [10]:
# a simple custom collate function, just to show the idea
def collatefn(batch):
    maxlen = max([l['emb'].shape[0] for l in batch])
    embdim = batch[0]['emb'].shape[1]
    withlabel = 'labels' in batch[0]
    if withlabel:
        labdim= batch[0]['labels'].shape[1]
        
    for b in batch:
        masklen = maxlen-len(b['emb'])
        b['emb'] = np.vstack((np.zeros((masklen, embdim)), b['emb']))
        b['embidx'] = torch.cat((torch.ones((masklen),dtype=torch.long)*-1, b['embidx']))
        b['mask'] = np.ones((maxlen))
        b['mask'][:masklen] = 0.
        if withlabel:
            b['labels'] = np.vstack((np.zeros((maxlen-len(b['labels']), labdim)), b['labels']))
            
    outbatch = {'emb' : torch.tensor(np.vstack([np.expand_dims(b['emb'], 0) \
                                                for b in batch])).float()}  
    outbatch['mask'] = torch.tensor(np.vstack([np.expand_dims(b['mask'], 0) \
                                                for b in batch])).float()
    outbatch['embidx'] = torch.tensor(np.vstack([np.expand_dims(b['embidx'], 0) \
                                                for b in batch])).float()
    if withlabel:
        outbatch['labels'] = torch.tensor(np.vstack([np.expand_dims(b['labels'], 0) for b in batch])).float()
    return outbatch

logger.info('Create loaders...')
trndataset = IntracranialDataset(trndf, trnemb, labels=True)
trnloader = DataLoader(trndataset, batch_size=batch_size, shuffle=True, num_workers=8, collate_fn=collatefn)

valdataset = IntracranialDataset(valdf, valemb, labels=False)
tstdataset = IntracranialDataset(tstdf, tstemb, labels=False)

tstloader = DataLoader(tstdataset, batch_size=batch_size*4, shuffle=False, num_workers=8, collate_fn=collatefn)
valloader = DataLoader(valdataset, batch_size=batch_size*4, shuffle=False, num_workers=8, collate_fn=collatefn)


# https://www.kaggle.com/bminixhofer/speed-up-your-rnn-with-sequence-bucketing
class NeuralNet(nn.Module):
    def __init__(self, embed_size=trnemb.shape[-1]*3, LSTM_UNITS=64, DO = 0.3):
        super(NeuralNet, self).__init__()
        
        self.embedding_dropout = SpatialDropout(0.0) #DO)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)
        self.linear2 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)

        self.linear = nn.Linear(LSTM_UNITS*2, n_classes)

    def forward(self, x, lengths=None):
        h_embedding = x

        h_embadd = torch.cat((h_embedding[:,:,:2048], h_embedding[:,:,:2048]), -1)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        h_conc_linear1  = F.relu(self.linear1(h_lstm1))
        h_conc_linear2  = F.relu(self.linear2(h_lstm2))
        
        hidden = h_lstm1 + h_lstm2 + h_conc_linear1 + h_conc_linear2 + h_embadd

        output = self.linear(hidden)
        
        return output

logger.info('Create model')
model =     NeuralNet(LSTM_UNITS=LSTM_UNITS, DO = DROPOUT)
model = model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
plist = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': DECAY},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = optim.Adam(plist, lr=lr)
scheduler = StepLR(optimizer, 1, gamma=lrgamma, last_epoch=-1)
model, optimizer = amp.initialize(model, optimizer, opt_level="O1")



2020-04-28 17:17:05,962 - Rsna-lstm-0-1-fp16 - INFO - Create loaders...
2020-04-28 17:17:06,618 - Rsna-lstm-0-1-fp16 - INFO - Create model


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [18]:
ypredls = []
ypredtstls = []
if not  os.path.exists(WORK_DIR + '/weights'):
    os.mkdir(WORK_DIR + '/weights')
for epoch in range(EPOCHS):
    tr_loss = 0.
    for param in model.parameters():
        param.requires_grad = True
    model.train()  
    for step, batch in enumerate(trnloader):
        y = batch['labels'].to(device, dtype=torch.float)
        mask = batch['mask'].to(device, dtype=torch.int)
        x = batch['emb'].to(device, dtype=torch.float)
        x = torch.autograd.Variable(x, requires_grad=True)
        y = torch.autograd.Variable(y)
        logits = model(x).to(device, dtype=torch.float)
        # get the mask for masked labels
        maskidx = mask.view(-1)==1
        y = y.view(-1, n_classes)[maskidx]
        logits = logits.view(-1, n_classes)[maskidx]
        # Get loss
        loss = criterion(logits, y)
        
        tr_loss += loss.item()
        optimizer.zero_grad()
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        optimizer.step()
        if step%1000==0:
            logger.info('Trn step {} of {} trn lossavg {:.5f}'. \
                        format(step, len(trnloader), (tr_loss/(1+step))))
    output_model_file = os.path.join(WORK_DIR, 'weights/lstm_gepoch{}_lstmepoch{}_fold{}.bin'.format(GLOBALEPOCH, epoch, fold))
    torch.save(model.state_dict(), output_model_file)

    scheduler.step()
    model.eval()
    '''
    logger.info('Prep val score...')
    ypred, imgval = predict(valloader)
    ypredls.append(ypred)
     
    yvalpred = sum(ypredls[-nbags:])/len(ypredls[-nbags:])
    yvalout = makeSub(yvalpred, imgval)
    yvalp = makeSub(ypred, imgval)
    
    # get Val score
    weights = ([1, 1, 1, 1, 1, 2] * ypred.shape[0])
    yact = valloader.dataset.data[label_cols].values#.flatten()
    yact = makeSub(yact, valloader.dataset.data['Image'].tolist())
    yact = yact.set_index('ID').loc[yvalout.ID].reset_index()
    valloss = log_loss(yact['Label'].values, yvalp['Label'].values.clip(.00001,.99999) , sample_weight = weights)
    vallossavg = log_loss(yact['Label'].values, yvalout['Label'].values.clip(.00001,.99999) , sample_weight = weights)
    logger.info('Epoch {} val logloss {:.5f} bagged {:.5f}'.format(epoch, valloss, vallossavg))
    '''
    logger.info('Prep test sub...')
    ypred, imgtst = predict(tstloader)
    ypredtstls.append(ypred)
    


2020-04-28 18:25:52,064 - Rsna-lstm-0-1-fp16 - INFO - Trn step 0 of 3909 trn lossavg 0.14773
2020-04-28 18:28:31,349 - Rsna-lstm-0-1-fp16 - INFO - Trn step 1000 of 3909 trn lossavg 0.06730


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0


2020-04-28 18:31:11,145 - Rsna-lstm-0-1-fp16 - INFO - Trn step 2000 of 3909 trn lossavg 0.06708
2020-04-28 18:33:50,746 - Rsna-lstm-0-1-fp16 - INFO - Trn step 3000 of 3909 trn lossavg 0.06701
2020-04-28 18:36:25,616 - Rsna-lstm-0-1-fp16 - INFO - Prep test sub...
2020-04-28 18:37:10,909 - Rsna-lstm-0-1-fp16 - INFO - Trn step 0 of 3909 trn lossavg 0.03425
2020-04-28 18:39:50,247 - Rsna-lstm-0-1-fp16 - INFO - Trn step 1000 of 3909 trn lossavg 0.06477


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0


2020-04-28 18:42:29,583 - Rsna-lstm-0-1-fp16 - INFO - Trn step 2000 of 3909 trn lossavg 0.06584
2020-04-28 18:45:09,279 - Rsna-lstm-0-1-fp16 - INFO - Trn step 3000 of 3909 trn lossavg 0.06529


...

2020-04-28 20:11:09,411 - Rsna-lstm-0-1-fp16 - INFO - Trn step 1000 of 3909 trn lossavg 0.04060
2020-04-28 20:13:58,412 - Rsna-lstm-0-1-fp16 - INFO - Trn step 2000 of 3909 trn lossavg 0.04016
2020-04-28 20:16:47,778 - Rsna-lstm-0-1-fp16 - INFO - Trn step 3000 of 3909 trn lossavg 0.04087
2020-04-28 20:19:25,269 - Rsna-lstm-0-1-fp16 - INFO - Prep test sub...
2020-04-28 20:19:46,255 - Rsna-lstm-0-1-fp16 - INFO - Trn step 0 of 3909 trn lossavg 0.05971
2020-04-28 20:22:35,504 - Rsna-lstm-0-1-fp16 - INFO - Trn step 1000 of 3909 trn lossavg 0.03645
2020-04-28 20:25:24,523 - Rsna-lstm-0-1-fp16 - INFO - Trn step 2000 of 3909 trn lossavg 0.03616


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0


2020-04-28 20:28:13,416 - Rsna-lstm-0-1-fp16 - INFO - Trn step 3000 of 3909 trn lossavg 0.03617
2020-04-28 20:30:53,421 - Rsna-lstm-0-1-fp16 - INFO - Prep test sub...
2020-04-28 20:31:18,520 - Rsna-lstm-0-1-fp16 - INFO - Trn step 0 of 3909 trn lossavg 0.03559


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0


2020-04-28 20:34:07,371 - Rsna-lstm-0-1-fp16 - INFO - Trn step 1000 of 3909 trn lossavg 0.03041
2020-04-28 20:36:56,037 - Rsna-lstm-0-1-fp16 - INFO - Trn step 2000 of 3909 trn lossavg 0.03013
2020-04-28 20:39:45,823 - Rsna-lstm-0-1-fp16 - INFO - Trn step 3000 of 3909 trn lossavg 0.03036
2020-04-28 20:42:23,425 - Rsna-lstm-0-1-fp16 - INFO - Prep test sub...


In [21]:
if not  os.path.exists('../preds'):
    os.mkdir('../preds')

logger.info('Write out bagged prediction to preds folder')
ytstpred = sum(ypredtstls[-nbags:])/len(ypredtstls[-nbags:])
ytstout = makeSub(ytstpred, imgtst)
ytstout.to_csv('../preds/lstm{}{}{}_{}_epoch{}_sub_{}.csv.gz'.format(TTAHFLIP, TTATRANSPOSE, LSTM_UNITS, WORK_DIR.split('/')[-1], epoch, embnm), \
            index = False, compression = 'gzip')

2020-04-29 07:28:34,595 - Rsna-lstm-0-1-fp16 - INFO - Write out bagged prediction to preds folder
